---
title: "USDT/BOB"
subtitle: "Historico del precio del USDT (Tether US Dollar) contra el BOB (Boliviano)"
author: 
  - name: Andres Chirinos
    email: ahchirinos@umsa.bo
  - name: Kevin Barrientos
lang: es
format: 
  dashboard:
    logo: "images/logo.webp"
    scrolling: true
    expandable: true
    nav-buttons:
      - github
      - text: Articulo
        icon: file-text-fill
        href: article\index.qmd
    theme: cosmo
execute:
  enabled: true
  freeze: true
---

# Proyecto migrado a [P2P BOB Exchange](https://github.com/andres-chirinos/p2p-bob-exchange)
Este repositorio esta archivado y ya no se le dara soporte. El nuevo repositorio es el siguiente: [P2P BOB Exchange](https://github.com/andres-chirinos/p2p-bob-exchange)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import pytz
from plotly.offline import init_notebook_mode, iplot

# Inicializar el modo de notebook
# init_notebook_mode(connected=True)
# Añade esto al inicio de tu notebook
# %matplotlib inline
# pio.renderers.default = 'vscode'

# Load the data
trade_df = pd.read_csv("data/currency_exchange_rates.csv")

# Asegurarse de que las columnas necesarias estén presentes
required_columns = ["price","available","advertisers_qty","type","timestamp","curr_from","curr_to","source"]
for col in required_columns:
    if col not in trade_df.columns:
        raise ValueError(f"Missing required column: {col}")

trade_df = trade_df.reset_index()
gmt_minus_4 = pytz.timezone('Etc/GMT-4')
trade_df['timestamp'] = pd.to_datetime(trade_df['timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert(gmt_minus_4)

# Filtrar datos de compra y venta desde el punto de vista del comerciante
buy_df = trade_df.query("type == 'SELL'")
buy_df = buy_df.groupby(buy_df['timestamp']).agg(
    price=('price', lambda x: x.quantile(0.10)),
    quantity=('available', 'sum'),
).reset_index()

sell_df = trade_df.query("type == 'BUY'")
sell_df = sell_df.groupby(sell_df['timestamp']).agg(
    price=('price', lambda x: x.quantile(0.90)),
    quantity=('available', 'sum'),
).reset_index()


In [2]:
# Calcular OHLC (Open, High, Low, Close) y volumen diario para compra y venta
buy_ohlc_df = buy_df.groupby(buy_df['timestamp'].dt.date).agg(
    high=('price', lambda x: max(x.max(), x.iloc[0])),  # Precio más alto del día o el precio de apertura
    low=('price', lambda x: min(x.min(), x.iloc[0])),   # Precio más bajo del día o el precio de apertura
    close=('price', lambda x: x.iloc[-1]),              # Precio más bajo del último registro del día
    volume=('quantity', 'mean')
).reset_index()

# Añadir el precio de cierre de ayer como 'open'
buy_ohlc_df['open'] = buy_ohlc_df['close'].shift(1)

# Reordenar columnas para mantener el formato OHLC
buy_ohlc_df = buy_ohlc_df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

# Calcular OHLC (Open, High, Low, Close) y volumen diario para compra y venta
sell_ohlc_df = sell_df.groupby(sell_df['timestamp'].dt.date).agg(
    high=('price', lambda x: max(x.max(), x.iloc[0])),  # Precio más alto del día o el precio de apertura
    low=('price', lambda x: min(x.min(), x.iloc[0], x.iloc[-1])),   # Precio más bajo del día o el precio de apertura
    close=('price', lambda x: x.iloc[-1]),              # Precio del último registro del día
    volume=('quantity', 'mean')
).reset_index()

# Añadir el precio de cierre de ayer como 'open'
sell_ohlc_df['open'] = sell_ohlc_df['close'].shift(1)

# Reordenar columnas para mantener el formato OHLC
sell_ohlc_df = sell_ohlc_df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

## Row {.small}

In [3]:
#| content: valuebox
#| title: "Precio de Compra de Hoy"
#| icon: currency-dollar
#| color: green
dict(
  value = buy_ohlc_df["close"].iloc[-1]
)

{'value': 11.102}

In [4]:
#| content: valuebox
#| title: "Precio de Venta de Hoy"
#| icon: currency-dollar
#| color: red
dict(
  value = sell_ohlc_df["close"].iloc[-1]
)

{'value': 10.964}

In [5]:
#| content: valuebox
#| title: "Volumen Total de Hoy"
#| icon: currency-dollar
#| color: secondary
dict(
  value = buy_ohlc_df["volume"].iloc[-1] + sell_ohlc_df["volume"].iloc[-1]
)

{'value': 1543504.1166666667}

## Row {.full}

In [6]:
# Crear el gráfico de líneas interactivo
fig = px.line()

# Añadir datos de compra (mínimo precio)
fig.add_scatter(x=buy_df['timestamp'], y=buy_df['price'], mode='lines', name='Compra', line=dict(color='blue'))

# Añadir datos de venta (máximo precio)
fig.add_scatter(x=sell_df['timestamp'], y=sell_df['price'], mode='lines', name='Venta', line=dict(color='red'))

# Añadir el gráfico de velas para compra
fig.add_trace(go.Candlestick(
    x=buy_ohlc_df['timestamp'],
    open=buy_ohlc_df['open'],
    high=buy_ohlc_df['high'],
    low=buy_ohlc_df['low'],
    close=buy_ohlc_df['close'],
    name='Precio de Compra',
    increasing_line_color='blue',
    decreasing_line_color='blue'
))

# Añadir el gráfico de velas para venta
fig.add_trace(go.Candlestick(
    x=sell_ohlc_df['timestamp'],
    open=sell_ohlc_df['open'],
    high=sell_ohlc_df['high'],
    low=sell_ohlc_df['low'],
    close=sell_ohlc_df['close'],
    name='Precio de Venta',
    increasing_line_color='red',
    decreasing_line_color='red'
))

# Añadir el volumen de compra
fig.add_trace(go.Bar(
    x=buy_ohlc_df['timestamp'],
    y=buy_ohlc_df['volume'],
    name='Volumen de Compra',
    marker_color='blue',
    opacity=0.3,
    yaxis='y2'
))

# Añadir el volumen de venta
fig.add_trace(go.Bar(
    x=sell_ohlc_df['timestamp'],
    y=sell_ohlc_df['volume'],
    name='Volumen de Venta',
    marker_color='red',
    opacity=0.3,
    yaxis='y2'
))

# Añadir una línea fija en el valor 6.96
fig.add_shape(
    type="line",
    x0=buy_ohlc_df['timestamp'].min(),
    y0=6.96,
    x1=buy_ohlc_df['timestamp'].max(),
    y1=6.96,
    line=dict(
        color="green",
        width=2,
        dash="dash",
    ),
    name='Línea Fija 6.96'
)

# Configurar el diseño del gráfico
fig.update_layout(
    title='Gráfico de Líneas Interactivo de Precios de Compra y Venta',
    xaxis_title='Fecha y Hora',
    yaxis_title='Precio (USDT)',
    legend_title='Tipo de Comercio',
    autosize=True,
    yaxis2=dict(
        title='Volumen',
        overlaying='y',
        side='right'
    )
)


# Mostrar el gráfico
fig.show()

## Row {.small}

In [7]:
ultima_actualizacion = trade_df['timestamp'].max()
print(ultima_actualizacion)

2024-12-10 05:51:39+04:00
